Modelling seal population sizes with Pyro
==========================================

> Pipeline to model detection error for CNN output and estimate seal haul out patterns and population sizes using environmental covariates. The model works accross two scales: patch (spatial scale for seal habitat use); and subpatch (spatial scale for seal haulout behavior). Besides being biologically inspired, some of the covariate data are extract from course resolution sensors (e.g. MODIS), thus separating between patch and subpatch scale prevents pseudoreplication at the subpatch level. Pyro is a probabilistic programming language developed by Uber using PyTorch as its backend. 


Setup
-------------------------

> Import a few packages, set random seed for reproducibility and enable output validation. 

In [38]:
import os
import torch
import pyro
import pandas as pd
from torch.distributions import constraints
from pyro import distributions as dist
from pyro.distributions.util import broadcast_shape
from pyro.infer import Trace_ELBO, TraceEnum_ELBO, config_enumerate
from pyro.infer.autoguide import AutoDiagonalNormal
import pyro.poutine as poutine
from pyro.optim import Adam
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
from functools import partial

pyro.set_rng_seed(1)

logging.basicConfig(format='%(message)s', level=logging.INFO)

# Enable validation checks -- throws errors when output gets outside of distribution support
pyro.enable_validation(__debug__)
smoke_test = ('CI' in os.environ)
pyro.set_rng_seed(1)

Generating fake data
------------------------

> Generate fake data for training. A few global variables decide how many patches we are using and how many subpatches within each patch. Covariates are drawn at random from suitable distributions and are then normalized to facilitate learning by gradient descent. 

### Covariate list

#### Patch (MODIS, 250m):

1. Chlorophyl concentration
2. Distance to continental shelf break
3. Julian Day
4. Time of the day
5. Sea ice concentration

#### Subpatch (WV03, 0.3m):

1. Floe size
2. Sea ice concentration

#### Observed variables (Subpatch):

1. Observed seals on ice (*N_obs*)
2. True number of seals on ice (*N_ice*)

#### Additional covariates?
* Past season covariate values
* Species occurrence (krill, seabirds, whales...)

### *TODO :*
* determine scale for patches and subpatches
* add correlations to fake data (sample with multivariate Normal)
* extract real environmental covariates and get high-res sea ice data
* test with real covariates

In [26]:

# number of patches
N_rows = 100

# number of subpatches per patch
N_cols = 3

# number of patch covariates
N_covs_patch = 5

# number of subpatch covariates
N_covs_subp = 2

# observed and true
data = pd.DataFrame({
    'obs': [dist.Poisson(5).sample([N_cols]) for _ in range(N_rows)],
    'true': [dist.Poisson(5).sample([N_cols]) for _ in range(N_rows)]
})

# currently all fake covariates are comming from a Gamma distribution, as if they were normalized. 
for i in range(N_covs_patch):
    data[f"cov_patch_{i}"] = [dist.Gamma(1, 1).sample([1]) for _ in range(N_rows)]
for j in range(N_covs_subp):
    data[f"cov_subp_{j}"] = [dist.Gamma(1, 1).sample([N_cols]) for _ in range(N_rows)]

Define model in Pyro
-----

> Model definition includes model hierarchy and distribution of choice for each parameter. 

### Model equations

##### Deterministic:
$$ \lambda_{total[i]} = Softplus(a_{\lambda total} * X_{patch[i]} + b_{\lambda total})$$

$$ \lambda_{fp[i, j]} = Softplus(a_{\lambda fp} * X_{sub[i, j]} + b_{\lambda fp})$$

$$ \alpha_{haul patch[i]} = Softplus(a_{\alpha haul patch} * X_{patch[i]} + b_{\alpha haul patch})$$

$$ \beta_{haul patch[i]} = Softplus(a_{\beta haul patch} * X_{patch[i]} + b_{\beta haul patch})$$

$$ (\rho_{sub [i, 0]} ... \rho_{sub [i, j]}) = \dfrac{Softplus(a_{\rho sub} * (X_{sub[i, 0]} ... X_{sub[i, j]}))}{|Softplus(a_{\rho sub} * (X_{sub[i, 0]} ... X_{sub[i, j]}))|}$$



##### Probabilistic:

$$ N_{total[i]} \sim Poisson(\lambda_{total[i]})$$

$$ \phi_{haul patch[i]} \sim Beta(\alpha_{haul patch[i]}, \beta_{haul patch[i]}) $$

$$ (\phi_{haul sub[i, 0]} ... \phi_{haul sub[i, j]}) \sim Dirichlet(\rho_{sub [i, 0]} ... \rho_{sub [i, j]}) $$

$$ (N_{ice[i, 0]} ...  N_{ice[i, j]}) \sim Multinomial(N_{total[i]} * \phi_{haul patch[i]}, \phi_{haul sub[i]})$$

$$ FalsePositives_{[i, j]} \sim Poisson(\lambda_{fp[i, j]}) $$

$$ N_{det[i, j]} \sim Binomial \left((N_{ice[i, j]} + \dfrac{FalsePositives_{[i, j]}}{\phi_{det[i, j]}},  \phi_{det[i, j]} \right) $$  

### A few words on Pyro

* Pyro supports PyTorch CUDA capibilities, parallelizing computations on GPUs. 
* Pyro 'plates' are wrappers for parallelized for loops, need to carefully specify input dimensions.
* While Pyro supports MCMC samplers, they use Hamiltonian Monte-Carlo, which does not support discrete latent variables. Since we are modelling counts, we will turn to Stochastic Variational Inference (SVI), which assumes a distribution for all posteriors and learns parameter values through gradient descent with an ELBO loss. 
* Apart from a model, we need to define a 'guide'. The guide gives proposal distributions for all sampled parameters in the model, which will be used during SVI.  

### *TODO :*
* Test a few different model formulations.

In [72]:
@poutine.broadcast
def model_fun1(data):
    # softplus transform as a link function
    softplus = torch.nn.Softplus()

    # extract patch and subpatch data
    patch_cols = [idx for idx, ele in enumerate(data.columns) if 'patch' in ele]
    subp_cols = [idx for idx, ele in enumerate(data.columns) if 'subp' in ele]
    cov_patch = data.iloc[:, patch_cols]
    cov_subp = data.iloc[:, subp_cols]

    # unroll covariate data to Tensors
    gt_N_ice = torch.Tensor([ele.cpu().numpy() for ele in data.true])
    gt_N_obs = torch.Tensor([ele.cpu().numpy() for ele in data.obs])
    cov_p = torch.randn([N_rows, 1, N_covs_patch])
    cov_s = torch.randn([N_rows, N_cols, N_covs_subp])
    for i in range(N_covs_patch):
        cov_p[:, :, i] = torch.Tensor([ele.cpu().numpy() for ele in cov_patch.iloc[:, i]])
    for i in range(N_covs_subp):
        cov_s[:, :, i] = torch.Tensor([ele.cpu().numpy() for ele in cov_subp.iloc[:, i]])

    
    # parameter names -- patch
    patch_par_names = []
    for par in ['a_', 'b_']:
        patch_par_names.extend([
            par + ele for ele in ['lambda_total', 'alpha_haul_prob_patch', 'beta_haul_prob_patch']
        ])

    # parameter names -- subpatch
    subp_par_names = []
    for par in ['a_', 'b_']:
        subp_par_names.extend([
            par + ele for ele in [
                'lambda_false_pos', 'alpha_haul_prob_subp', 'beta_haul_prob_subp', 'alpha_det',
                'beta_det'
            ]
        ])
    
    # parameter starting values
    alphas_patch = pyro.param('alphas_patch', torch.Tensor([1] * len(patch_par_names)), constraint=constraints.positive)
    betas_patch = pyro.param('betas_patch', torch.Tensor([1] * len(patch_par_names)), constraint=constraints.positive)
    alphas_subp = pyro.param('alphas_subp', torch.Tensor([1] * len(subp_par_names)), constraint=constraints.positive)
    betas_subp = pyro.param('betas_subp', torch.Tensor([1] * len(subp_par_names)), constraint=constraints.positive)

    patch_params = {
        ele: pyro.sample(
            ele,
            dist.Gamma(alphas_patch[idx], betas_patch[idx]).expand([N_covs_patch]).independent(1))
        for idx, ele in enumerate(patch_par_names)
    }

    
    subp_params = {
        ele: pyro.sample(
            ele,
            dist.Gamma(alphas_subp[idx], betas_subp[idx]).expand([N_covs_subp]).independent(1))
        for idx, ele in enumerate(subp_par_names)
    }

    # create plates for parallelizing
    x = pyro.plate('x', size=N_rows, dim=-2)
    y = pyro.plate('y', size=N_cols, dim=-1)

    # patch loop
    with x:
        # deterministic linear functions
        lambda_total = softplus(
            torch.sum(patch_params['a_lambda_total'] * cov_p + patch_params['b_lambda_total']))
        alpha_haul_prob_patch = softplus(
            torch.sum(patch_params['a_alpha_haul_prob_patch'] * cov_p +
                      patch_params['b_alpha_haul_prob_patch']))
        beta_haul_prob_patch = softplus(
            torch.sum(patch_params['a_beta_haul_prob_patch'] * cov_p +
                      patch_params['b_beta_haul_prob_patch']))

        # draw haul out probability for patches
        haul_prob_patch = pyro.sample('haul_prob_patch',
                                      dist.Beta(alpha_haul_prob_patch, beta_haul_prob_patch))

        # get total number of seals for patches
        N_total = pyro.sample('N_total', dist.Poisson(lambda_total))
        
        # get totao number of hauled out seals
        N_total_haul = pyro.sample('N_total_haul', dist.Binomial(N_total, haul_prob_patch))

    # subpatch loop
    with x, y:
        # deterministic linear functions
        lambda_false_pos = softplus(
            torch.sum(subp_params['a_lambda_false_pos'] * cov_s +
                      subp_params['b_lambda_false_pos']))
        alpha_haul_prob_subp = softplus(
            torch.sum(subp_params['a_alpha_haul_prob_subp'] * cov_s +
                      subp_params['b_alpha_haul_prob_subp']))
        beta_haul_prob_subp = softplus(
            torch.sum(subp_params['a_beta_haul_prob_subp'] * cov_s +
                      subp_params['b_beta_haul_prob_subp']))
        alpha_det_subp = softplus(
            torch.sum(subp_params['a_alpha_det'] * cov_s + subp_params['b_alpha_det']))
        beta_det_subp = softplus(
            torch.sum(subp_params['a_beta_det'] * cov_s + subp_params['b_beta_det']))

        # draw haul out probability for subpatches (subpatch specific)
        haul_prob_subp = pyro.sample('haul_prob_subp',
                                     dist.Gamma(alpha_haul_prob_subp, beta_haul_prob_subp))
        det_prob_subp = pyro.sample('det_subp', dist.Beta(alpha_det_subp, beta_det_subp))
        false_pos = pyro.sample('false_pos', dist.Poisson(lambda_false_pos))
        

    if observe:
        for i in pyro.plate('rows', N_rows):
            N_ice = pyro.sample(f'N_ice_{i}',
                                dist.DirichletMultinomial(concentration=haul_prob_subp[i, :],
                                                          total_count=N_total_haul[i],
                                                          validate_args=False),
                                obs=gt_N_ice[i, :])
            for j in pyro.plate(f'cols_{i}', N_cols):
                pyro.sample(f'N_obs_{i}_{j}',
                            dist.Binomial(total_count=max(0, (N_ice[j] +
                                                       false_pos[i, j] / det_prob_subp[i, j]).int()),
                                          probs=det_prob_subp[i, j]),
                            obs=gt_N_obs[i, j])

Train model
----

> We use an Adam optimizer to train our model for a given number of steps. We define a function to inspect optimization.

### *TODO :*
* Add support for training on mini-batches
* Check why values for **N_obs** fall outside of support sometimes
* Check why model is sensible to starting values

In [85]:
pyro.clear_param_store()


# peek at parameter values
def print_progress():
    alphas = pyro.param("alphas_patch")
    betas = pyro.param("betas_patch")
    print(f"SVI steps: {step}")
    print(f"   elbo loss: {svi.evaluate_loss(data)}")
    print(f"   alpha parameter values for patch covariates: {list(alphas.detach().numpy())}")
    print(f"   beta parameter values for patch covariates : {list(betas.detach().numpy())} \n")


adam_params = {"lr": 0.001}
optimizer = Adam(adam_params)
model = partial(model_fun1, observe=True)
guide = partial(model_fun1, observe=False)
elbo = Trace_ELBO(max_plate_nesting=2)

svi = SVI(model, guide, optimizer, elbo)

n_steps = 2000
for step in range(n_steps):
    svi.step(data)
    if step % 100 == 0:
        print_progress()



SVI steps: 0
   elbo loss: 318927.5104980469
   alpha parameter values for patch covariates: [0.9990005, 0.9990005, 1.0010005, 0.9990005, 0.9990005, 1.0010005]
   beta parameter values for patch covariates : [1.0010005, 1.0010005, 0.9990005, 1.0010005, 1.0010005, 0.9990005] 

SVI steps: 100
   elbo loss: 292443.8864746094
   alpha parameter values for patch covariates: [1.0097847, 1.0118084, 0.98816884, 1.0122502, 1.011182, 0.99024564]
   beta parameter values for patch covariates : [0.9921404, 0.98921543, 1.0132442, 0.988011, 0.9888353, 1.0090166] 

SVI steps: 200
   elbo loss: 187110.59118652344
   alpha parameter values for patch covariates: [1.008764, 1.0112083, 0.9841876, 1.0162002, 1.0125357, 0.99235225]
   beta parameter values for patch covariates : [0.99468166, 0.9895773, 1.0186106, 0.98297614, 0.9882638, 1.0050309] 

SVI steps: 300
   elbo loss: 1617535.8310546875
   alpha parameter values for patch covariates: [1.0072486, 1.0140495, 0.9843799, 1.017069, 1.0167009, 0.98852146

ValueError: Error while computing log_prob at site 'N_obs_30_2':
The value argument must be within the support
               Trace Shapes:           
                Param Sites:           
                alphas_patch        6  
                 betas_patch        6  
                 alphas_subp       10  
                  betas_subp       10  
               Sample Sites:           
         a_lambda_total dist        | 5
                       value        | 5
                    log_prob        |  
a_alpha_haul_prob_patch dist        | 5
                       value        | 5
                    log_prob        |  
 a_beta_haul_prob_patch dist        | 5
                       value        | 5
                    log_prob        |  
         b_lambda_total dist        | 5
                       value        | 5
                    log_prob        |  
b_alpha_haul_prob_patch dist        | 5
                       value        | 5
                    log_prob        |  
 b_beta_h

Results
----

In [ ]:
svi.